# Analyzing RCT data with Precision Adjustment

In [ ]:
import pandas as pd
import numpy as np
import patsy
import statsmodels.formula.api as smf
import statsmodels.api as sm

## Data

In this lab, we analyze the Pennsylvania re-employment bonus experiment, which was previously studied in "Sequential testing of duration data: the case of the Pennsylvania ‘reemployment bonus’ experiment" (Bilias, 2000), among others. These experiments were conducted in the 1980s by the U.S. Department of Labor to test the incentive effects of alternative compensation schemes for unemployment insurance (UI). In these experiments, UI claimants were randomly assigned either to a control group or one of five treatment groups. Actually, there are six treatment groups in the experiments. Here we focus on treatment group 4, but feel free to explore other treatment groups. In the control group the current rules of the UI applied. Individuals in the treatment groups were offered a cash bonus if they found a job within some pre-specified period of time (qualification period), provided that the job was retained for a specified duration. The treatments differed in the level of the bonus, the length of the qualification period, and whether the bonus was declining over time in the qualification period; see http://qed.econ.queensu.ca/jae/2000-v15.6/bilias/readme.b.txt for further details on data. 
  

In [ ]:
data = pd.read_csv("https://raw.githubusercontent.com/VC2015/DMLonGitHub/master/penn_jae.dat", delim_whitespace=True)
n, p = data.shape
data = data[data["tg"].isin([0, 4])]

In [ ]:
data["T4"] = np.where(data["tg"] == 4, 1, 0)
data["T4"].value_counts()

In [ ]:
data.describe()

### Model 
To evaluate the impact of the treatments on unemployment duration, we consider the linear regression model:

$$
Y =  D \beta_1 + W'\beta_2 + \varepsilon, \quad E \varepsilon (D,W')' = 0,
$$

where $Y$ is  the  log of duration of unemployment, $D$ is a treatment  indicators,  and $W$ is a set of controls including age group dummies, gender, race, number of dependents, quarter of the experiment, location within the state, existence of recall expectations, and type of occupation.   Here $\beta_1$ is the ATE, if the RCT assumptions hold rigorously.


We also consider interactive regression model:

$$
Y =  D \alpha_1 + D W' \alpha_2 + W'\beta_2 + \varepsilon, \quad E \varepsilon (D,W', DW')' = 0,
$$
where $W$'s are demeaned (apart from the intercept), so that $\alpha_1$ is the ATE, if the RCT assumptions hold rigorously.

Under RCT, the projection coefficient $\beta_1$ has
the interpretation of the causal effect of the treatment on
the average outcome. We thus refer to $\beta_1$ as the average
treatment effect (ATE). Note that the covariates, here are
independent of the treatment $D$, so we can identify $\beta_1$ by
just linear regression of $Y$ on $D$, without adding covariates.
However we do add covariates in an effort to improve the
precision of our estimates of the average treatment effect.

### Analysis

We consider 

*  classical 2-sample approach, no adjustment (CL)
*  classical linear regression adjustment (CRA)
*  interactive regression adjusment (IRA)

# Carry out covariate balance check

This is done using "cov_type='HC1'" which computes the correct Eicher-Huber-White standard errors, instead of the classical non-robust formula based on the homoscdedasticity.

In [ ]:
covariate_balance = smf.ols("T4~(female+black+othrace+C(dep)+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)**2",
                            data)
covariate_balance.fit(cov_type="HC1").summary()

We see that that even though this is a randomized experiment, balance conditions are failed.

# Model Specification

In [ ]:
# no adjustment (2-sample approach)
cl = smf.ols("np.log(inuidur1) ~ T4", data)

# adding controls
cra = smf.ols("np.log(inuidur1) ~ T4 + (female+black+othrace+C(dep)+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)**2",
              data)
# Omitted dummies: q1, nondurable, muld

cl_results = cl.fit(cov_type="HC1")
cl_results.summary()

In [ ]:
cra_results = cra.fit(cov_type="HC1")
cra_results.summary()

The interactive specification corresponds to the approach introduced in Lin (2013).

In [ ]:
#interactive regression model

ira_formula = "0 + (female+black+othrace+C(dep)+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)**2"
X = patsy.dmatrix(ira_formula, data, return_type='dataframe')
X.columns = [f'x{t}' for t in range(X.shape[1])] # clean column names
X = (X - X.mean(axis=0)) # demean all control covariates

# construct interactions of treatment and (de-meaned covariates, 1)
ira_formula = "T4 * ("+ "+".join(X.columns) + ")"
X['T4'] = data['T4']
X = patsy.dmatrix(ira_formula, X, return_type='dataframe')
ira = sm.OLS(np.log(data[["inuidur1"]]), X)
ira_results = ira.fit(cov_type="HC1")
ira_results.summary()

Next we try out partialling out with lasso

In [ ]:
from sklearn.linear_model import LassoCV

D = X['T4']
A = X.drop('T4', axis=1) # contains X and interaction terms D*X
y = np.log(data["inuidur1"])

Dres = D - np.mean(D)
yres = y - LassoCV(max_iter=10000).fit(A, y).predict(A)

ira_lasso_results = sm.OLS(yres, Dres).fit(cov_type='HC1')
ira_lasso_results.summary()

### Results

In [ ]:
results = {
    "CL": [cl_results.params[cl.exog_names.index("T4")], 
           cl_results.HC1_se[cl.exog_names.index("T4")]],
    "CRA": [cra_results.params[cra.exog_names.index("T4")], 
           cra_results.HC1_se[cra.exog_names.index("T4")]],
    "IRA": [ira_results.params[ira.exog_names.index("T4")], 
           ira_results.HC1_se[ira.exog_names.index("T4")]],
    "IRA w Lasso": [ira_lasso_results.params[0], 
           ira_lasso_results.bse[0]],
}
results = pd.DataFrame(results)
results.index = ["Estimate", "Standard Error"]
results
# for printing to LaTeX: print(results.to_latex())

Treatment group 4 experiences an average decrease of about $7.8\%$ in the length of unemployment spell.


Observe that regression estimators delivers estimates that are slighly more efficient (lower standard errors) than the simple 2 mean estimator, but essentially all methods have very similar standard errors. From IRA results we also see that there is not any statistically detectable heterogeneity.  We also see the regression estimators offer slightly lower estimates -- these difference occur perhaps to due minor imbalance in the treatment allocation, which the regression estimators try to correct.


